# Join Expressions

It determines whether Spark should bring together the left set of data with right set of data 

# Join Types 

It Determines what should be in the result set 

- Inner Joins 
- Outer Joins
- Left Outer Joins
- Right Outer Joins
- Left Semi Joins
- Left Anti Joins
- Natural Joins
- Cross Joins

# Create Spark Session

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Avi').master('local').getOrCreate()

23/03/11 18:03:56 WARN Utils: Your hostname, avijit-HP-Laptop-15q-bu0xx resolves to a loopback address: 127.0.1.1; using 192.168.18.7 instead (on interface wlo1)
23/03/11 18:03:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/11 18:04:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# # Create Dataframes required for the illustration purposes

In [2]:
person_df = spark.createDataFrame(
[
    (0,"Avijit",0,[100]),
    (1,"Avik",1,[500,250,100]),
    (2,"Sudipto",1,[250,100])
]
).toDF("id","name","graduate_program","spark_status")

graduate_program_df = spark.createDataFrame(
[
    (0,"Masters","Mathematics","IIT Indore"),
    (2,"Masters","Physics","IIT Indore"),
    (1,"Ph.d","Physics","IIT Indore")
]).toDF("id","degree","department","school")

spark_status_df = spark.createDataFrame([
    (500,"Vice President"),
    (250,"PMC Member"),
    (100,"Contributor")
    
]).toDF("id","Status")

# Register these as sql tables


In [3]:
person_df.createOrReplaceTempView("person_table")
graduate_program_df.createOrReplaceTempView("graduate_program_table")
spark_status_df.createOrReplaceTempView("spark_status_table")

# Inner Join

It evaluates keys in both dataframe or tables and include only rows that evaluate to true

In [4]:
spark.sql("""
SELECT * FROM person_table
INNER JOIN graduate_program_table
ON person_table.graduate_program = graduate_program_table.id

""").show()

+---+-------+----------------+---------------+---+-------+-----------+----------+
| id|   name|graduate_program|   spark_status| id| degree| department|    school|
+---+-------+----------------+---------------+---+-------+-----------+----------+
|  0| Avijit|               0|          [100]|  0|Masters|Mathematics|IIT Indore|
|  1|   Avik|               1|[500, 250, 100]|  1|   Ph.d|    Physics|IIT Indore|
|  2|Sudipto|               1|     [250, 100]|  1|   Ph.d|    Physics|IIT Indore|
+---+-------+----------------+---------------+---+-------+-----------+----------+



# Outer Join
It evaluate the keys in both of the dataframes or tables and includes the rows that evaluate to true or false

In [5]:
spark.sql("""
SELECT * FROM person_table
FULL OUTER JOIN graduate_program_table
ON person_table.graduate_program = graduate_program_table.id

""").show()

+----+-------+----------------+---------------+---+-------+-----------+----------+
|  id|   name|graduate_program|   spark_status| id| degree| department|    school|
+----+-------+----------------+---------------+---+-------+-----------+----------+
|   0| Avijit|               0|          [100]|  0|Masters|Mathematics|IIT Indore|
|   1|   Avik|               1|[500, 250, 100]|  1|   Ph.d|    Physics|IIT Indore|
|   2|Sudipto|               1|     [250, 100]|  1|   Ph.d|    Physics|IIT Indore|
|null|   null|            null|           null|  2|Masters|    Physics|IIT Indore|
+----+-------+----------------+---------------+---+-------+-----------+----------+



# Left Outer Join
evaluate the keys in both dataframes or tables and includes all rows from the left dataframe as well as any rows in the right dataframe that have a match in the left dataframe


In [6]:
spark.sql("""
SELECT * FROM person_table
LEFT OUTER JOIN graduate_program_table
ON person_table.graduate_program = graduate_program_table.id

""").show()

+---+-------+----------------+---------------+---+-------+-----------+----------+
| id|   name|graduate_program|   spark_status| id| degree| department|    school|
+---+-------+----------------+---------------+---+-------+-----------+----------+
|  0| Avijit|               0|          [100]|  0|Masters|Mathematics|IIT Indore|
|  1|   Avik|               1|[500, 250, 100]|  1|   Ph.d|    Physics|IIT Indore|
|  2|Sudipto|               1|     [250, 100]|  1|   Ph.d|    Physics|IIT Indore|
+---+-------+----------------+---------------+---+-------+-----------+----------+



# Right Outer Join
evaluate the keys in both dataframes or tables and includes all rows from the right dataframe as well as any rows in the left dataframe that have a match in the right dataframe


In [8]:
spark.sql("""
SELECT * FROM person_table
RIGHT OUTER JOIN graduate_program_table
ON person_table.graduate_program = graduate_program_table.id

""").show()

+----+-------+----------------+---------------+---+-------+-----------+----------+
|  id|   name|graduate_program|   spark_status| id| degree| department|    school|
+----+-------+----------------+---------------+---+-------+-----------+----------+
|   0| Avijit|               0|          [100]|  0|Masters|Mathematics|IIT Indore|
|   2|Sudipto|               1|     [250, 100]|  1|   Ph.d|    Physics|IIT Indore|
|   1|   Avik|               1|[500, 250, 100]|  1|   Ph.d|    Physics|IIT Indore|
|null|   null|            null|           null|  2|Masters|    Physics|IIT Indore|
+----+-------+----------------+---------------+---+-------+-----------+----------+



#  Left Semi Join

Semi joins are bit of departure from other joins. They do not actually include any values from the right dataframe. They only compare values to see if the values exist in the right dataframe

In [9]:
spark.sql("""
SELECT * FROM person_table
LEFT SEMI JOIN graduate_program_table
ON person_table.graduate_program = graduate_program_table.id

""").show()

+---+-------+----------------+---------------+
| id|   name|graduate_program|   spark_status|
+---+-------+----------------+---------------+
|  0| Avijit|               0|          [100]|
|  1|   Avik|               1|[500, 250, 100]|
|  2|Sudipto|               1|     [250, 100]|
+---+-------+----------------+---------------+



#  Left Anti Join

Left Anti Joins are opposite of left semi joins. Rather than keeping the values that exist in second dataframe,
they keep only values that do not have corresponding key in second dataframe

In [10]:
spark.sql("""
SELECT * FROM person_table
LEFT ANTI JOIN graduate_program_table
ON person_table.graduate_program = graduate_program_table.id

""").show()

+---+----+----------------+------------+
| id|name|graduate_program|spark_status|
+---+----+----------------+------------+
+---+----+----------------+------------+



In [11]:
spark.sql("""
SELECT * FROM graduate_program_table
LEFT ANTI JOIN person_table
ON person_table.graduate_program = graduate_program_table.id

""").show()

+---+-------+----------+----------+
| id| degree|department|    school|
+---+-------+----------+----------+
|  2|Masters|   Physics|IIT Indore|
+---+-------+----------+----------+



# Natural Joins
It makes implicit guess on the columns on which you would like to join. It finds the matching columns and returns the results 

# Cross Joins
In simplest terms cross joins are inner joins that do not specify a predicate

In [17]:
spark.sql("""
SELECT * FROM person_table
CROSS JOIN graduate_program_table
""").show()

+---+-------+----------------+---------------+---+-------+-----------+----------+
| id|   name|graduate_program|   spark_status| id| degree| department|    school|
+---+-------+----------------+---------------+---+-------+-----------+----------+
|  0| Avijit|               0|          [100]|  0|Masters|Mathematics|IIT Indore|
|  0| Avijit|               0|          [100]|  2|Masters|    Physics|IIT Indore|
|  0| Avijit|               0|          [100]|  1|   Ph.d|    Physics|IIT Indore|
|  1|   Avik|               1|[500, 250, 100]|  0|Masters|Mathematics|IIT Indore|
|  1|   Avik|               1|[500, 250, 100]|  2|Masters|    Physics|IIT Indore|
|  1|   Avik|               1|[500, 250, 100]|  1|   Ph.d|    Physics|IIT Indore|
|  2|Sudipto|               1|     [250, 100]|  0|Masters|Mathematics|IIT Indore|
|  2|Sudipto|               1|     [250, 100]|  2|Masters|    Physics|IIT Indore|
|  2|Sudipto|               1|     [250, 100]|  1|   Ph.d|    Physics|IIT Indore|
+---+-------+---

In [16]:
spark.sql("""
SELECT * FROM person_table
CROSS JOIN graduate_program_table
ON person_table.graduate_program = graduate_program_table.id

""").show()

+---+-------+----------------+---------------+---+-------+-----------+----------+
| id|   name|graduate_program|   spark_status| id| degree| department|    school|
+---+-------+----------------+---------------+---+-------+-----------+----------+
|  0| Avijit|               0|          [100]|  0|Masters|Mathematics|IIT Indore|
|  1|   Avik|               1|[500, 250, 100]|  1|   Ph.d|    Physics|IIT Indore|
|  2|Sudipto|               1|     [250, 100]|  1|   Ph.d|    Physics|IIT Indore|
+---+-------+----------------+---------------+---+-------+-----------+----------+

